# Day 23

倒数第三天的问题很有点意思，需要技巧实现，不然时间复杂度将会无法接受。两个部分考察的都是循环链表的实现，这也是为数不多真正需要用到链表才能体现性能优越性的场景了。

## Part I

第一部分按照题目意思，将所有的杯子形成一个循环链表即可，由于杯子数n很小，因此再寻找将移出链表的三个杯子重新入链的位置的时候，并不需要技巧，依次找一遍所有的杯子，直到找到合乎条件的杯子即可。

首先定义杯子Cup类和循环链表CupsCircle类：

In [1]:
class Cup(object):
    def __init__(self, cup_id: int):
        self.cup_id = cup_id
        # 链表下一个
        self.next = None
        
    def __repr__(self) -> str:
        return str(self.cup_id)

class CupsCircle(object):
    def __init__(self, data: str):
        '''构建杯子组成的循环链表'''
        prev_cup = None
        for i, cup_id in enumerate(data):
            cup = Cup(int(cup_id))
            if i == 0:
                # 初始化当前杯子指针 current
                self.current = cup
            if prev_cup:
                # 将上一个杯子的next指向当前杯子
                prev_cup.next = cup
            # 迭代下一个杯子
            prev_cup = cup
        # 最后一个杯子的next指向第一个杯子current，完成循环链构建
        prev_cup.next = self.current
    
    def take_tripple(self) -> Cup:
        '''取出当前杯子后的三个杯子：脱链'''
        tripple_begin = self.current.next
        tripple_end = self.current.next.next.next
        self.current.next = tripple_end.next
        tripple_end.next = None
        return tripple_begin
    
    def find_destination(self) -> Cup:
        '''找到插入脱链杯子的位置'''
        current_id = self.current.cup_id
        # 与当前杯子ID的最小间隔
        min_gap = 1 << 31
        # 最大的杯子ID以及对应的杯子
        max_id = -1
        max_id_cup = None
        # 目标位置杯子，初始化为None
        destination_cup = None
        # 下面迭代所有杯子，找到符合条件的位置，记录到destination_cup中
        cup_ptr = self.current.next
        while cup_ptr != self.current:
            gap = current_id - cup_ptr.cup_id
            # 如果ID相差1，直接返回该位置
            if gap == 1:
                return cup_ptr
            # 记录最小间隔位置及相应杯子
            if gap > 0 and gap < min_gap:
                min_gap = gap
                destination_cup = cup_ptr
            # 记录最大ID及相应杯子
            if gap < 0 and cup_ptr.cup_id > max_id:
                max_id = cup_ptr.cup_id
                max_id_cup = cup_ptr
            cup_ptr = cup_ptr.next
        # 返回符合条件的位置
        return destination_cup if destination_cup else max_id_cup
    
    def put_tripple_back(self, dest: Cup, tripple: Cup):
        '''将三个杯子放回链表中：入链'''
        tripple_end = tripple.next.next
        tripple_end.next = dest.next
        dest.next = tripple
    
    def one_move(self):
        '''一轮游戏，出链-》找位置-》入链'''
        tripple = self.take_tripple()
        destination = self.find_destination()
        self.put_tripple_back(destination, tripple)
        self.current = self.current.next
    
    def no_one_serie(self) -> str:
        '''完成游戏后，在链表中找到所有ID为1之后的杯子ID的序列，返回字符串'''
        cup_ptr = self.current
        # 找到ID为1的杯子
        while cup_ptr.cup_id != 1:
            cup_ptr = cup_ptr.next
            cup_ptr = cup_ptr.next
        cup_ptr = cup_ptr.next
        result = ''
        # 再次遍历，直到ID为1为止，输出ID字符串序列
        while cup_ptr.cup_id != 1:
            result += str(cup_ptr)
            cup_ptr = cup_ptr.next
        return result

第一部分的逻辑：

In [2]:
def part1_solution(data: str, moves: int) -> str:
    circle = CupsCircle(data)
    for _ in range(moves):
        circle.one_move()
    return circle.no_one_serie()

单元测试：

In [3]:
assert(part1_solution('389125467', 10) == '92658374')

In [4]:
assert(part1_solution('389125467', 100) == '67384529')

第一部分结果：

In [5]:
part1_solution('364297581', 100)

'47382659'

## Part II

一百万的杯子！！游戏进行一千万次！！！！

很显然，这就是将第一部分的时间复杂度增加了百亿倍。。。我们首先来看看按照第一部分的方法计算9个杯子一万轮游戏的时间：

In [6]:
%timeit part1_solution('364297581', 10000)

37.8 ms ± 614 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


将近40毫秒，而第二部分的复杂度将是这个时间的一亿倍，简单估算一下，大约需要46天，肯定不行，这样是算不出来的。

于是必须想出一个新的算法来完成第二部分，首先我们应该知道，这里的时间复杂度主要是由于寻找插入位置那个步骤造成的，那里需要遍历整一个链表来找到合适的位置插入三个杯子，而我们肯定有办法能够避免。下面我们重新定义Cup和CupsCircle类，这里的想法就是，在所有杯子中，再加入一个循环链表，这个循环链表永远是ID倒序方向的，比如三个杯子的情况，那就是3=>2=>1=>3...无论我们将哪三个杯子从原来的链表中脱离，这个链都保持不变，然后我们就可以从当前位置current沿着新的这个链表向后（倒序）查找，只要找到第一个仍在链中的杯子，那就是插入位置，我们通过新增的这个链表，可以将每次查找的时间复杂度由o(1000000)降低到O(3)（注意两个o大小写不一样），这样我们最终的时间复杂度就仅仅是游戏进行一千万次的耗时了。

下面就是重定义的类：

In [7]:
class Cup(object):
    def __init__(self, cup_id: int):
        self.cup_id = cup_id
        self.next = None
        # 新的链表下一项，less_one表示倒序，实际上是循环链表，这个名称也许不是特别合适
        self.less_one = None
        # 加入一个状态属性，表示杯子是否处于圈（原链表）中
        self.is_chained = True
        
    def __repr__(self) -> str:
        return str(self.cup_id)

class CupsCircle(object):
    def __init__(self, data: str, end: int=1_000_001):
        '''初始化一百万个杯子的两个链表'''
        prev_cup = None
        # 记录前面有标注标号杯子的字典，用于生成倒序链表的前面部分
        cups_dict = {}
        for i, cup_id in enumerate(data):
            cup = Cup(int(cup_id))
            if i == 0:
                self.current = cup
            if prev_cup:
                prev_cup.next = cup
            if cup_id == '1':
                # 记录ID为1的杯子，方便获得第二部分结果
                self.first_cup = cup
            cups_dict[int(cup_id)] = cup
            prev_cup = cup
        # 迭代字典生成前面输入数据中的杯子的倒序链表
        for cup_id, cup in cups_dict.items():
            if cup_id > 1:
                cup.less_one = cups_dict[cup_id - 1]
        # 输入数据中最大ID的杯子
        less_one_cup = cups_dict[len(data)]
        # 生成一百万个里面剩下杯子的链表
        for cup_id in range(len(data) + 1, end):
            cup = Cup(cup_id)
            # 圈链表
            prev_cup.next = cup
            # 倒序链表
            cup.less_one = less_one_cup
            prev_cup = cup
            less_one_cup = cup
        # 形成圈，最后一个杯子的next指向第一个杯子current
        prev_cup.next = self.current
        # 倒序循环链表，1号杯子的倒序为最后一个杯子
        self.first_cup.less_one = less_one_cup
    
    def take_tripple(self) -> Cup:
        '''出链'''
        tripple_begin = self.current.next
        cup_ptr = tripple_begin
        for _ in range(2):
            # 设置杯子状态为出链
            cup_ptr.is_chained = False
            cup_ptr = cup_ptr.next
        cup_ptr.is_chained = False
        self.current.next = cup_ptr.next
        cup_ptr.next = None
        return tripple_begin
    
    def find_destination(self) -> Cup:
        '''寻找合适位置插入，增加一个链表后，不止性能大幅提升，整个逻辑也更清晰简单'''
        cup_ptr = self.current.less_one
        # 直到找到一个仍在圈中的杯子为止
        while not cup_ptr.is_chained:
            cup_ptr = cup_ptr.less_one
        return cup_ptr
    
    def put_tripple_back(self, dest: Cup, tripple: Cup):
        '''入链'''
        cup_ptr = tripple
        for _ in range(2):
            # 设置杯子状态入链
            cup_ptr.is_chained = True
            cup_ptr = cup_ptr.next
        cup_ptr.is_chained = True
        cup_ptr.next = dest.next
        dest.next = tripple
    
    def one_move(self):
        '''游戏一轮，逻辑不变'''
        tripple = self.take_tripple()
        destination = self.find_destination()
        self.put_tripple_back(destination, tripple)
        self.current = self.current.next

第二部分的逻辑：

In [8]:
def part2_solution(circle: CupsCircle, moves: int=10_000_000) -> int:
    for _ in range(moves):
        circle.one_move()
    return circle.first_cup.next.cup_id * circle.first_cup.next.next.cup_id

我们先来对比一下使用第二部分逻辑分别对9个杯子和一百万个杯子进行10000轮游戏的时间：

In [9]:
circle = CupsCircle('389125467', 10)
%timeit part2_solution(circle, 10000)

32.4 ms ± 1.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
circle = CupsCircle('389125467')
%timeit part2_solution(circle, 10000)

31.6 ms ± 130 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


相当棒，使用一百万个杯子进行游戏甚至比使用9个杯子还要快！当然Python跑起来还是比较吃力，但是方法没问题。

然后是单元测试：

In [11]:
circle = CupsCircle('389125467')
assert(part2_solution(circle) == 149245887792)

最后是第二部分的结果：

In [12]:
circle = CupsCircle('364297581')
part2_solution(circle)

42271866720